<a href="https://colab.research.google.com/github/Nobel712/Image-Segmentation/blob/main/Unet%2BECA%2BDenseNet121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import backend as K

def ECANet(input_tensor):
    x = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('sigmoid')(x)
    return x

def eca_densenet(input_shape=(256, 256, 3), num_classes=1):
    input_tensor = Input(input_shape)

    # DenseNet121 encoder
    densenet = DenseNet121(include_top=False, input_tensor=input_tensor)
    encoder_output = densenet.get_layer('relu').output

    # ECANet module
    eca_output = ECANet(encoder_output)

    # U-Net decoder
    up = UpSampling2D(size=(32, 32))(eca_output)  # Modify the upsampling size
    up = Conv2D(128, (3, 3), padding='same')(up)  # Reduce the number of filters
    up = BatchNormalization()(up)
    up = Activation('relu')(up)
    up = Conv2D(128, (3, 3), padding='same')(up)  # Reduce the number of filters
    up = BatchNormalization()(up)
    up = Activation('relu')(up)

    # Final prediction
    output = Conv2D(num_classes, (1, 1), activation='sigmoid')(up)

    model = Model(inputs=input_tensor, outputs=output)
    return model

# Create the model
model = eca_densenet()
model.summary()